#Бибилиотеки

In [ ]:
#Установим необходимые библиотеки

!pip install scikit-surprise

!pip install implicit

!pip install lightfm

In [ ]:
#Загрузим необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import scipy.sparse as sparse
from surprise.model_selection import train_test_split

import implicit
import pickle

from lightfm import LightFM

#Подключимся к диску с данными

In [ ]:
# Если работаем через google disk
# Подключимся к диску с данными
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Директория с датасетом
dir_path ='/content/gdrive/MyDrive/Final_work_3_DS/data_recommender_system'

In [ ]:
#Загрузим датасет с товарами по юзерам из директории (должен быть посчитан заранее)
df_counter = pd.read_csv(dir_path + '/counter_df')

In [ ]:
#Создадим функцию для категоризации данных
def make_df_cat(df_c, path_df):
  df = df_c.copy()

  df['user_id'] = df['user_id'].astype('category')
  df['product_id'] = df['product_id'].astype('category')
  df['count'] = df['count'].astype('category')

  df['user'] = df['user_id'].cat.codes
  df['product'] = df['product_id'].cat.codes
  df['count'] = df['count'].cat.codes

  df.to_csv(path_df, index=False)

In [ ]:
make_df_cat(df_counter, dir_path + '/df_with_cat')

In [ ]:
df_cat = pd.read_csv(dir_path + '/df_with_cat')

In [ ]:
#Создадим spare-matrix для работы с алгоритмами рекомендательных систем
item_user_matrix = sparse.csr_matrix(
    (df_cat['count'].astype(float),
     (df_cat['product'], df_cat['user']))
    )
    
user_item_matrix = sparse.csr_matrix(
    (df_cat['count'].astype(float),
     (df_cat['user'], df_cat['product']))
    )

In [ ]:
df_cat

,user_id,product_id,count,user,product
0,1,196,9,0,195
1,1,12427,9,0,12375
2,1,10258,8,0,10216
3,1,25133,7,0,25032
4,1,46149,2,0,45950
...,...,...,...,...,...
9459060,206209,755,0,99999,752
9459061,206209,890,0,99999,885
9459062,206209,6825,0,99999,6792
9459063,206209,44325,0,99999,44136


#ALS

Воспользуемся алгоритмом Alternating Least Squares (ALS) библиотеки implicit

In [ ]:
als_model = implicit.als.AlternatingLeastSquares(factors=1000, 
                                                 regularization=0.0, 
                                                 iterations=10)

In [ ]:
als_model.fit(user_item_matrix)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
#df для декодирования продуктов
uncode_pr = pd.DataFrame()
uncode_pr['product'] = df_cat['product'].unique()
uncode_pr['product_id'] = df_cat['product_id'].unique()
uncode_pr.index = df_cat['product'].unique()

In [ ]:
#Сформируем файл посмотрим на результат
submission_df = pd.DataFrame()
submission_df['user_id'] = df_transact.user_id.unique()
submission_df.index = df_cat.user.unique()
submission_df['product_id'] = None

for i in tqdm(df_cat.user.unique()):
  prediction = als_model.recommend(i, 
                                   item_user_matrix, 
                                   N=10, 
                                   filter_already_liked_items=False, 
                                   recalculate_user=False)[0].tolist()
  pred = []
  for n in prediction:
    id_pr = uncode_pr.at[n, 'product_id']
    pred.append(id_pr)
  pred = [str(x) for x in pred]
  submission_df.at[i, 'product_id'] = " ".join(pred)

100%|██████████| 100000/100000 [03:28<00:00, 478.54it/s]


In [ ]:
#Сохраним итоговые рекомендации на диск
submission_df.to_csv(dir_path + '/submission.csv', sep=',', index=False)

Результаты работы модели с кодировкой признаков (MAP@10):

**Score: 0.10343**
(factors=30, iterations=10)


**Score: 0.13921**
(factors=150, iterations=10)


**Score: 0.14145**
(factors=200, iterations=10)


**Score: 0.14227**
(factors=300, iterations=10)


**Score: 0.14503**
(factors=500, iterations=10)


**Score: 0.14768**
(factors=1000, iterations=10)


**Score: 0.0006**
(factors=2000, iterations=10)


**Score: 0.0006**
(factors=5000, iterations=10)


**Score: 0.0006**
(factors=10000, iterations=10)